<a href="https://colab.research.google.com/github/CodeCraftIA/Market-Basket-Analysis/blob/main/ErgasiaEE1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 1qklC6HBqynRmhn5q-zY3Ov2a0jcCwT9s

**Άσκηση** 1η

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
data = pd.read_csv('/content/GroceriesInitial.csv', header = None)

In [ ]:
data.head()

**Μετασχηματισμός προτογενών δεδομένων σε δυαδικά**

In [ ]:

#needed data
ndata = ["citrus fruit", "tropical fruit", "whole milk", "other vegetables", "rolls/buns", "chocolate", "bottled water", "yogurt",
"sausage", "root vegetables", "pastry", "sodα", "cream"]

nonItemVal=["id","basket_value","recency_days"]
transacts=[]
xData=nonItemVal+ndata
#γεμίζω με 0 όλα τα στοιχεία του πίνακα(μόνο τα 13 προιόντα για κάθε συναλαγή)
for i in range(0, 7536):
  transacts.append([0 for i in range(0,16)])
# βάζω και τα στοιχεία ("id","basket_value","recency_days")
for i in range(0, 7536):
  for j in range(0, 3):
    transacts[i][j]=data.values[i+1,j]
# βάζω 1 κάτω απο κάθε στήλη (προιόν) που υπάρχει στην συναλλαγή
for i in range(0, 7536):
    for n in range(0,13):
      if ndata[n] in str(data.values[i+1]):
        transacts[i][n+3]=1
pd.DataFrame(transacts,columns=xData)





Διακριτοποίηση του **Basket_value**

In [ ]:
xData2=[]
#transacts[i][1] εκει βρίσκονται οι τιμες basket_value
for i in range(0,7536):
  xData2.append(float(transacts[i][1]))
#τις ταξινομώ
xData2.sort()
count1=0
count2=0
count3=0
#μετράω το πλήθος κάθε κατηγορίας
for i in range(0,7536):
  if xData2[i]<3:
    count1=count1+1
  elif xData2[i]<6.5:
    count2=count2+1
  else:
    count3=count3+1
print("low_value_basket :",count1)
print("medium_value_basket :",count2)
print("high_value_basket :",count3)
#αλλάζω την αριθμητική τιμη με την διακριτή στην οποία ανήκει
for i in range(0, 7536):
  if float(transacts[i][1])<3:
     transacts[i][1]="low_value_basket"
  elif float(transacts[i][1])<6.5:
      transacts[i][1]="medium_value_basket"
  else:
      transacts[i][1]="high_value_basket"
pd.DataFrame(transacts,columns=xData)


**2η Άσκηση**

In [ ]:
#ετοιμάζω τα στοιχεία τα οποία χρειάζομαι(μονο τα items) για να εφαρμόσω apriory
itemsData=[]
lineData=[]
for i in range(0, 7536):
  for j in range(0, 13):
    if transacts[i][j+3]==1:
      lineData.append(ndata[j])
  itemsData.append(lineData)
  lineData=[]
for i in range(0, 7536):
  print(itemsData[i])



In [ ]:
!pip install apyori

In [ ]:
#α ερώτημα , απλά τρέχω τον αλγόριθμο αλλάζοντας τις τιμές του support
from apyori import apriori
rule = apriori(transactions = itemsData, min_support = 0.3, min_confidence = 0.00001)# min_lift = 3, min_length = 2, max_length = 2)
list(rule)

In [ ]:
#Συνάρτηση για καλύτερη οπτικοποίηση των αποτελεσμάτων Apriory καθώς και ταξινόμησή τους
def orderInspect(results,orderBy):
  df = pd.DataFrame(columns=('Items-data','Antecedent','Consequent','Support','Confidence','Lift'))

  Support =[]
  Confidence = []
  Lift = []
  Items = []
  Antecedent = []
  Consequent=[]

  for RelationRecord in results:
      for ordered_stat in RelationRecord.ordered_statistics:
          Support.append(RelationRecord.support)
          Items.append(RelationRecord.items)
          Antecedent.append(ordered_stat.items_base)
          Consequent.append(ordered_stat.items_add)
          Confidence.append(ordered_stat.confidence)
          Lift.append(ordered_stat.lift)

  df['Items-data'] = list(map(set, Items))
  df['Antecedent'] = list(map(set, Antecedent))
  df['Consequent'] = list(map(set, Consequent))
  df['Support'] = Support
  df['Confidence'] = Confidence
  df['Lift']= Lift
  df.sort_values(by =orderBy, ascending = False, inplace = True)
  return df


In [ ]:
#β ερώτημα
from apyori import apriori
rule = apriori(transactions = itemsData, min_support = 0.001, min_confidence = 0.001)# min_lift = 3, min_length = 2, max_length = 2)
#list(rule)
results = list(rule)
out=orderInspect(results,'Confidence')
out

In [ ]:
#εκτός απο τα items data πλεον χρειάζομαι και το basket value για το γ ερώτημα
data2=[]
lineData=[]
for i in range(0, 7536):
  lineData.append(transacts[i][1])##+basketvalue
  for j in range(0, len(itemsData[i])):
      lineData.append(itemsData[i][j])
  data2.append(lineData)
  lineData=[]
for i in range(0, 7536):
  print(data2[i])

In [ ]:
rule = apriori(transactions = data2, min_support = 0.001, min_confidence = 0.001)# min_lift = 3, min_length = 2, max_length = 2)
#list(rule)
results = list(rule)
out=orderInspect(results,'Confidence')
out

**3η Άσκηση**

In [ ]:
list3=[]#η λιστα η οποία θα περιέχει τα basketvalue,recencydays
lineData=[]
recd=[]#recencydays
baskv=[]#basketvalue
for i in range(0, 7536):
  lineData.append(float(data.values[i+1,1]))
  lineData.append(float(data.values[i+1,2]))
  list3.append(lineData)
  lineData=[]
for i in range(0, 7536):
  baskv.append(float(data.values[i+1,1]))
  recd.append(float(data.values[i+1,2]))
for i in range(0, 7536):
  print(list3[i])


**Δημιουργία 5 ομάδων μέσω Kmeans**

In [ ]:
from sklearn.cluster import KMeans
#kmeans = KMeans(n_clusters=5)
#kmeans.fit(list3)
#y_kmeans = kmeans.predict(list3)
#print(y_kmeans)
kmeans = KMeans(n_clusters=5, random_state=0).fit(list3)
y_kmeans = kmeans.predict(list3)
kmeans.labels_


In [ ]:
plt.scatter(baskv, recd, c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter([centers[0][0],centers[1][0],centers[2][0],centers[3][0],centers[4][0]], [centers[0][1],centers[1][1],centers[2][1],centers[3][1],centers[4][1]], c='black', s=200, alpha=0.5);

**Υπολογισμός μέσης τιμής και τυπικής απόκλισης**

In [ ]:
from numpy.ma.extras import average
import statistics
print(centers[0])#cluster0(mwv)
print(centers[1])#cluster1(ble)
print(centers[2])#cluster2(galazio)
print(centers[3])#cluster3(prasino)
print(centers[4])#cluster4(kitrino)
x=average([centers[0][0],centers[1][0],centers[2][0],centers[3][0],centers[4][0]])
y=average([centers[0][1],centers[1][1],centers[2][1],centers[3][1],centers[4][1]])
print("Μέση τιμή κέντρων: ")
print([x,y])
x2=statistics.stdev([centers[0][0],centers[1][0],centers[2][0],centers[3][0],centers[4][0]])
y2=statistics.stdev([centers[0][1],centers[1][1],centers[2][1],centers[3][1],centers[4][1]])
print("Τυπική απόκλιση κέντρων: ")
print([x2,y2])

**αριθμητικό προφίλ κάθε ομάδας**

In [ ]:
c0=0
c1=0
c2=0
c3=0
c4=0
for i in kmeans.labels_:
  if i==0:
    c0=c0+1
  elif i==1:
    c1=c1+1
  elif i==2:
    c2=c2+1
  elif i==3:
    c3=c3+1
  else:
    c4=c4+1
print("Cluster0 : Ομάδα πρόσφατων συναλλαγών κυρίως μεσαίας και μεγάλης αξίας (και ελαχίστων αρκετά μικρών) που αντιπροσωπεύει το ",(c0/7537)*100,"% επι του συνόλου")
print("Cluster1 : Ομάδα παλαιών συναλλαγών(περίπου 2 μηνες) απο μικρής έως μεσαίας αξίας που αντιπροσωπεύει το ",(c1/7537)*100,"% επι του συνόλου")
print("Cluster2 : Ομάδα  παλαιών συναλλαγών(περίπου 1 μήνα) απο μικρής έως μεσαίας αξίας που αντιπροσωπεύει το ",(c2/7537)*100,"% επι του συνόλου")
print("Cluster3 : Ομάδα παλαιότερων συναλλαγών(2+ μήνες) όλων των αξιών που αντιπροσωπεύει το ",(c3/7537)*100,"% επι του συνόλου")
print("Cluster4 : Ομάδα σχετικά πρόσφατων συναλλαγών(2-3 εβδομάδων) αποκλειστηκά μεσαίας και μεγάλης αξίας που αντιπροσωπεύει το ",(c4/7537)*100,"% επι του συνόλου")


**ομάδα της κάθε μιας συναλλαγής σε μια νεα ποιοτική μεταβλητή (στήλη)**

In [ ]:
xData=xData+["clusters"]
newCol=[]
for i in kmeans.labels_:
  if i==0:
    newCol.append("claster0")
  elif i==1:
    newCol.append("claster1")
  elif i==2:
    newCol.append("claster2")
  elif i==3:
    newCol.append("claster3")
  else:
    newCol.append("claster4")
for i in range(0,7536):
  transacts[i].append(newCol[i]) ##provlima to prostheti 3 fores

pd.DataFrame(transacts,columns=xData)


**4η Άσκηση**

In [ ]:
data4=[] #φτιάχνω μια λίστα με τα στοιχεία που χρειάζομαι
lineData=[]
for i in range(0, 7536):
  lineData.append(transacts[i][16])
  for j in range(0, len(itemsData[i])):
      lineData.append(itemsData[i][j])
  data4.append(lineData)
  lineData=[]
for i in range(0, 7536):
  print(data4[i])

**Α ερώτημα**

In [ ]:
rule = apriori(transactions = data4, min_support = 0.001, min_confidence = 0.001)# min_lift = 3, min_length = 2, max_length = 2)
#list(rule)
results = list(rule)
out=orderInspect(results,'Confidence')
out

**Β ερώτημα**

In [ ]:
data4b=[] #φτιάχνω μια λίστα με τα στοιχεία που χρειάζομαι
lineData=[]
for i in range(0, 7536):
  lineData.append(transacts[i][1])##+basketvalue
  for j in range(0, len(data4[i])):
      lineData.append(data4[i][j])
  data4b.append(lineData)
  lineData=[]
for i in range(0, 7536):
  print(data4b[i])

In [ ]:
rule = apriori(transactions = data4b, min_support = 0.001, min_confidence = 0.001)# min_lift = 3, min_length = 2, max_length = 2)
#list(rule)
results = list(rule)
out=orderInspect(results,'Confidence')
out